In [1]:
import math
from decimal import *
import numpy as np # linear algebra
import pandas as pd # read_csv and such
from geopy.distance import vincenty # calculate distance between locations
from io import StringIO # convert strings to buffers or something like that.
import os # for listing files in directory

pd.options.display.max_columns = 999

### File paths for data sets

In [2]:
# the paths to our flight related data sets
airlines_path = '../data/flight_delays_2015/airlines.csv'
airports_path = '../data/flight_delays_2015/airports.csv'
flights_path  = '../data/flight_delays_2015/flights.csv'
# weather related paths
weather_path = '../data/weather/2015.csv'
stations_path  = '../data/weather/stations.csv'
weather_files_path = '../data/weather/station_data/'
weather_merged_path = '../data/weather/weather_merged.csv'

# modified data sets for output
airports_modified_path = '../data/flight_delays_2015/airports_modified.csv'
flights_weather_path = '../data/flight_delays_2015/flights_weather.csv'


### Read in the data sets

In [3]:
# read the airlines in and get a dataframe containing the dataset
# Set low_memory to False so that types can be inferred/converted
airlines_df = pd.read_csv(airlines_path, low_memory=False) 
airlines_df.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [5]:
airports_df = pd.read_csv(airports_path, low_memory=False, dtype={'usaf': 'str', 'wban': 'str'})
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [4]:
flights_df = pd.read_csv(flights_path, low_memory=False, dtype={'SCHEDULED_DEPARTURE': 'str', 'DEPARTURE_TIME': 'str', 'WHEELS_OFF': 'str', 'WHEELS_ON': 'str', 'SCHEDULED_ARRIVAL': 'str', 'ARRIVAL_TIME': 'str'})
flights_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,2354,-11.0,21.0,0015,205.0,194.0,169.0,1448,0404,4.0,0430,0408,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,0002,-8.0,12.0,0014,280.0,279.0,263.0,2330,0737,4.0,0750,0741,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,0018,-2.0,16.0,0034,286.0,293.0,266.0,2296,0800,11.0,0806,0811,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,0015,-5.0,15.0,0030,285.0,281.0,258.0,2342,0748,8.0,0805,0756,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,0024,-1.0,11.0,0035,235.0,215.0,199.0,1448,0254,5.0,0320,0259,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
stations_df = pd.read_csv(stations_path, low_memory=False, dtype={'usaf': 'str', 'wban': 'str'})
stations_df.head()

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
0,007011,99999,CWOS 07011,NaN,NaN,NaN,NaN,NaN,NaN,20120101,20121129
1,007025,99999,CWOS 07025,NaN,NaN,NaN,NaN,NaN,NaN,20120127,20120127
2,007034,99999,CWOS 07034,NaN,NaN,NaN,NaN,NaN,NaN,20121024,20121106
3,007047,99999,CWOS 07047,NaN,NaN,NaN,NaN,NaN,NaN,20120613,20120717
4,007059,99999,CWOS 07059,NaN,NaN,NaN,NaN,NaN,NaN,20120314,20120828


### Add Date Column To Flights
Use Year, Month, Day columns to create datetime column that will be used later to match weather data.

In [5]:
flights_df['DATE'] = pd.to_datetime((flights_df.YEAR*10000+flights_df.MONTH*100+flights_df.DAY).apply(str),format='%Y%m%d')
flights_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,2354,-11.0,21.0,0015,205.0,194.0,169.0,1448,0404,4.0,0430,0408,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,0002,-8.0,12.0,0014,280.0,279.0,263.0,2330,0737,4.0,0750,0741,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,0018,-2.0,16.0,0034,286.0,293.0,266.0,2296,0800,11.0,0806,0811,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,0015,-5.0,15.0,0030,285.0,281.0,258.0,2342,0748,8.0,0805,0756,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,0024,-1.0,11.0,0035,235.0,215.0,199.0,1448,0254,5.0,0320,0259,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01


### Remove stations that are not in the US or are missing latitude/longitude
Since we know all our Airports are in the US, this will vastly improve performance on calculations

In [7]:
stations_df = stations_df[np.isfinite(stations_df['lat'])]
stations_df = stations_df[np.isfinite(stations_df['lon'])]
#stations_df = stations_df[stations_df['country'] == 'US']
stations_df.head()

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
16,010014,99999,SORSTOKKEN,NO,NaN,ENSO,59.792,5.341,48.8,19861120,20170915
17,010016,99999,RORVIK/RYUM,NO,NaN,NaN,64.850,11.233,14.0,19870116,19910806
18,010017,99999,FRIGG,NO,NaN,ENFR,59.980,2.250,48.0,19880320,20050228
19,010020,99999,VERLEGENHUKEN,NO,NaN,NaN,80.050,16.250,8.0,19861109,20170916
20,010030,99999,HORNSUND,NO,NaN,NaN,77.000,15.500,12.0,19850601,20170916


### Read in weather data for all the stations
Load all the station files and combine into a single dataframe

In [30]:
# skip if merged file is already created

# dateparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

# files = os.listdir(weather_files_path)
# weather_df = pd.read_fwf(weather_files_path + files[0], dtype={'STN---': 'str', 'WBAN': 'str', 'FRSHTT': 'str'}, parse_dates=['YEARMODA'], date_parser=dateparse)

# for i in  range(1, len(files)):
#     temp = pd.read_fwf(weather_files_path + files[i], dtype={'STN---': 'str', 'WBAN': 'str', 'FRSHTT': 'str'}, parse_dates=['YEARMODA'], date_parser=dateparse)
#     weather_df = pd.concat([weather_df, temp])

# weather_df.head()

,STN---,WBAN,YEARMODA,TEMP,Unnamed: 4,DEWP,Unnamed: 6,SLP,Unnamed: 8,STP,...,Unnamed: 12,WDSP,Unnamed: 14,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,483280,99999,2015-01-01,71.7,13,60.0,13,1018.2,8,989.9,...,13,1.5,13,6.0,999.9,84.2,59.9*,0.00I,999.9,000000
1,483280,99999,2015-01-02,71.2,15,57.1,15,1017.8,8,989.4,...,10,1.2,15,5.1,999.9,85.1,53.6*,0.00I,999.9,000000
2,483280,99999,2015-01-03,71.1,15,57.6,15,1016.7,8,988.3,...,15,0.7,15,4.1,999.9,86.0,53.6*,0.00I,999.9,000000
3,483280,99999,2015-01-04,72.9,14,59.7,14,1013.6,8,985.5,...,14,0.5,14,4.1,999.9,90.3,55.4*,0.00I,999.9,000000
4,483280,99999,2015-01-05,76.5,15,65.1,15,1010.4,8,982.5,...,11,0.5,15,2.9,999.9,91.6,59.0*,0.00I,999.9,000000


### Correct Columns
Some of the column names were not labled, merge the station and WBAN to create a single identifier, and drop the WBAN

In [31]:
# skip if merged file is already created

# weather_df = weather_df.rename(columns={'STN---': 'STATION', 'Unnamed: 4': 'COUNT_TEMP', 'Unnamed: 6': 'COUNT_DEWP', 'Unnamed: 8': 'COUNT_SLP', 'Unnamed: 10': 'COUNT_STP', 'Unnamed: 12': 'COUNT_VISIB', 'Unnamed: 14': 'COUNT_WDSP'})
# weather_df['STATION'] = weather_df['STATION'] + '-' + weather_df['WBAN']
# weather_df = weather_df.drop('WBAN', 1)
# weather_df.head()

,STATION,YEARMODA,TEMP,COUNT_TEMP,DEWP,COUNT_DEWP,SLP,COUNT_SLP,STP,COUNT_STP,...,COUNT_VISIB,WDSP,COUNT_WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,483280-99999,2015-01-01,71.7,13,60.0,13,1018.2,8,989.9,8,...,13,1.5,13,6.0,999.9,84.2,59.9*,0.00I,999.9,000000
1,483280-99999,2015-01-02,71.2,15,57.1,15,1017.8,8,989.4,8,...,10,1.2,15,5.1,999.9,85.1,53.6*,0.00I,999.9,000000
2,483280-99999,2015-01-03,71.1,15,57.6,15,1016.7,8,988.3,8,...,15,0.7,15,4.1,999.9,86.0,53.6*,0.00I,999.9,000000
3,483280-99999,2015-01-04,72.9,14,59.7,14,1013.6,8,985.5,8,...,14,0.5,14,4.1,999.9,90.3,55.4*,0.00I,999.9,000000
4,483280-99999,2015-01-05,76.5,15,65.1,15,1010.4,8,982.5,8,...,11,0.5,15,2.9,999.9,91.6,59.0*,0.00I,999.9,000000


### Save Dataframe
Save to a file to keep from having to run again.

In [32]:
# skip if merged file is already created

# weather_df.to_csv(weather_merged_path)

In [8]:
# load the file instead if you have merged file
weather_df = pd.read_csv(weather_merged_path, dtype={'STATION': 'str', 'FRSHTT': 'str', }, parse_dates=['YEARMODA'])
weather_df = weather_df.drop('Unnamed: 0', 1)
weather_df.head()

,Unnamed: 0,STATION,YEARMODA,TEMP,COUNT_TEMP,DEWP,COUNT_DEWP,SLP,COUNT_SLP,STP,COUNT_STP,VISIB,COUNT_VISIB,WDSP,COUNT_WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,0,483280-99999,2015-01-01,71.7,13,60.0,13,1018.2,8,989.9,8,5.4,13,1.5,13,6.0,999.9,84.2,59.9*,0.00I,999.9,000000
1,1,483280-99999,2015-01-02,71.2,15,57.1,15,1017.8,8,989.4,8,5.0,10,1.2,15,5.1,999.9,85.1,53.6*,0.00I,999.9,000000
2,2,483280-99999,2015-01-03,71.1,15,57.6,15,1016.7,8,988.3,8,3.7,15,0.7,15,4.1,999.9,86.0,53.6*,0.00I,999.9,000000
3,3,483280-99999,2015-01-04,72.9,14,59.7,14,1013.6,8,985.5,8,3.4,14,0.5,14,4.1,999.9,90.3,55.4*,0.00I,999.9,000000
4,4,483280-99999,2015-01-05,76.5,15,65.1,15,1010.4,8,982.5,8,4.6,11,0.5,15,2.9,999.9,91.6,59.0*,0.00I,999.9,000000


### Update Stations DataFrame to use same station id as weather data

In [10]:
stations_df = stations_df.rename(columns={'usaf': 'station'})
stations_df['station'] = stations_df['station'] + '-' + stations_df['wban']
stations_df = stations_df.drop('wban', 1)
stations_df.head()

,station,name,country,state,call,lat,lon,elev,begin,end
16,010014-99999,SORSTOKKEN,NO,NaN,ENSO,59.792,5.341,48.8,19861120,20170915
17,010016-99999,RORVIK/RYUM,NO,NaN,NaN,64.850,11.233,14.0,19870116,19910806
18,010017-99999,FRIGG,NO,NaN,ENFR,59.980,2.250,48.0,19880320,20050228
19,010020-99999,VERLEGENHUKEN,NO,NaN,NaN,80.050,16.250,8.0,19861109,20170916
20,010030-99999,HORNSUND,NO,NaN,NaN,77.000,15.500,12.0,19850601,20170916


### Add Data Counts to Stations
Update stations_df to also contain how many days of data that each station has.

In [11]:
station_counts_df = weather_df.groupby(['STATION']).count()[['TEMP']]
station_counts_df = station_counts_df.rename(columns={'TEMP': 'COUNT'})
station_counts_df.head()


,COUNT
STATION,
007070-99999,2
008411-99999,365
008414-99999,364
008415-99999,73
008416-99999,102


In [12]:
stations_df = pd.merge(stations_df, station_counts_df, left_on='station', right_index=True, how='inner')
stations_df.head()

,station,name,country,state,call,lat,lon,elev,begin,end,COUNT
16,010014-99999,SORSTOKKEN,NO,NaN,ENSO,59.792,5.341,48.8,19861120,20170915,301
19,010020-99999,VERLEGENHUKEN,NO,NaN,NaN,80.050,16.250,8.0,19861109,20170916,236
20,010030-99999,HORNSUND,NO,NaN,NaN,77.000,15.500,12.0,19850601,20170916,317
22,010060-99999,EDGEOYA,NO,NaN,NaN,78.250,22.817,14.0,19730101,20170916,292
23,010080-99999,LONGYEAR,SV,NaN,ENSB,78.246,15.466,26.8,19750929,20170916,365


### Assign each airport primary and secondary weather stations
Assign weather station based on closest weather station with data for full year


In [13]:
useful_stations_df = stations_df[stations_df['COUNT'] == 365]
len(useful_stations_df.station)

6202

In [16]:
for index, row in airports_df.iterrows():
    first = useful_stations_df.iloc[0]
    first_dist = vincenty((row['LATITUDE'], row['LONGITUDE']), (first['lat'], first['lon'])).miles
    second = useful_stations_df.iloc[1]
    second_dist = vincenty((row['LATITUDE'], row['LONGITUDE']), (second['lat'], second['lon'])).miles
    if second_dist < first_dist:
        first, first_dist, second, second_dist = second, second_dist, first, first_dist
    for i in range(2, len(useful_stations_df['station'])):
        dist = vincenty((row['LATITUDE'], row['LONGITUDE']), (useful_stations_df.iloc[i]['lat'], useful_stations_df.iloc[i]['lon'])).miles
        if dist < second_dist:
            if dist < first_dist:
                first, first_dist, second, second_dist = useful_stations_df.iloc[i], dist, first, first_dist
            else:
                second, second_dist = useful_stations_df.iloc[i], dist
    airports_df.loc[index, 'weather_station'], airports_df.loc[index, 'weather_station_distance'] = first['station'], first_dist
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,weather_station,weather_station_distance
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040,725170-14737,0.431330
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190,722660-13962,0.022812
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,723650-23050,0.405126
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183,726590-14929,0.599491
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447,722160-13869,0.043161


In [15]:
airports_df.sort_values(['weather_station_distance'], ascending=False)

,Unnamed: 0,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,weather_station,weather_station_distance
246,246,PPG,Pago Pago International Airport (Tafuna Airport),Pago Pago,AS,USA,14.331020,-170.710530,998337-99999,888.943123
137,137,GUM,Guam International Airport,Agana,GU,USA,13.483450,-144.795980,992620-99999,582.508518
9,9,ADK,Adak Airport,Adak,AK,USA,51.877960,-176.646030,992820-99999,359.698790
94,94,DVL,Devils Lake Regional Airport,Devils Lake,ND,USA,48.114250,-98.908780,727575-94925,70.642466
70,70,CNY,Canyonlands Field,Moab,UT,USA,38.754960,-109.754840,724760-23066,70.471628
248,248,PSE,Mercedita Airport,Ponce,PR,USA,18.008300,-66.563010,785263-11641,46.576880
295,295,STT,Cyril E. King Airport,Charlotte Amalie,VI,USA,18.337310,-64.973360,785510-11624,45.082374
318,318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.688400,-111.117640,726664-94173,35.775073
321,321,YUM,Yuma International Airport,Yuma,AZ,USA,32.656580,-114.605970,999999-53154,27.269475
102,102,ESC,Delta County Airport,Escanaba,MI,USA,45.722670,-87.093730,997792-99999,22.690501


### Distant Weather Stations
Some airports don't have a weather stations that are close and have data for the full year (18 that are more than 2 miles away)

### Time to Finally Join that Weather Data to the Flights

In [18]:
flights_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,...,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,...,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,...,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01


In [19]:
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,weather_station,weather_station_distance
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040,725170-14737,0.431330
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190,722660-13962,0.022812
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,723650-23050,0.405126
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183,726590-14929,0.599491
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447,722160-13869,0.043161


In [20]:
weather_df.head()

,STATION,YEARMODA,TEMP,COUNT_TEMP,DEWP,COUNT_DEWP,SLP,COUNT_SLP,STP,COUNT_STP,...,COUNT_VISIB,WDSP,COUNT_WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,483280-99999,2015-01-01,71.7,13,60.0,13,1018.2,8,989.9,8,...,13,1.5,13,6.0,999.9,84.2,59.9*,0.00I,999.9,000000
1,483280-99999,2015-01-02,71.2,15,57.1,15,1017.8,8,989.4,8,...,10,1.2,15,5.1,999.9,85.1,53.6*,0.00I,999.9,000000
2,483280-99999,2015-01-03,71.1,15,57.6,15,1016.7,8,988.3,8,...,15,0.7,15,4.1,999.9,86.0,53.6*,0.00I,999.9,000000
3,483280-99999,2015-01-04,72.9,14,59.7,14,1013.6,8,985.5,8,...,14,0.5,14,4.1,999.9,90.3,55.4*,0.00I,999.9,000000
4,483280-99999,2015-01-05,76.5,15,65.1,15,1010.4,8,982.5,8,...,11,0.5,15,2.9,999.9,91.6,59.0*,0.00I,999.9,000000


### Merge stations
Merge the airports_df on origin and destination airports to get stations for each flight.

In [14]:
flights_merged_df = pd.merge(flights_df, airports_df[['IATA_CODE', 'weather_station']], left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='left')
flights_merged_df = flights_merged_df.rename(columns={'weather_station': 'origin_weather_station'})
flights_merged_df = pd.merge(flights_merged_df, airports_df[['IATA_CODE', 'weather_station']], left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='left')
flights_merged_df = flights_merged_df.rename(columns={'weather_station': 'destination_weather_station'})
flights_merged_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,2354,-11.0,21.0,0015,205.0,194.0,169.0,1448,0404,4.0,0430,0408,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,0002,-8.0,12.0,0014,280.0,279.0,263.0,2330,0737,4.0,0750,0741,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,LAX,722950-23174,PBI,722030-12844
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,0018,-2.0,16.0,0034,286.0,293.0,266.0,2296,0800,11.0,0806,0811,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,SFO,724940-23234,CLT,723140-13881
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,0015,-5.0,15.0,0030,285.0,281.0,258.0,2342,0748,8.0,0805,0756,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,LAX,722950-23174,MIA,722020-12839
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,0024,-1.0,11.0,0035,235.0,215.0,199.0,1448,0254,5.0,0320,0259,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,SEA,727930-24233,ANC,702725-26491


### Merge the weather for origin airport
Merge on matching date and station.

In [15]:
flights_merged_df = pd.merge(flights_merged_df, weather_df, left_on=['origin_weather_station', 'DATE'], right_on=['STATION', 'YEARMODA'], how='left')
flights_merged_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,STATION,YEARMODA,TEMP,COUNT_TEMP,DEWP,COUNT_DEWP,SLP,COUNT_SLP,STP,COUNT_STP,VISIB,COUNT_VISIB,WDSP,COUNT_WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,2354,-11.0,21.0,0015,205.0,194.0,169.0,1448,0404,4.0,0430,0408,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233,702725-26491,2015-01-01,35.1,24.0,30.7,24.0,1020.1,14.0,1015.2,24.0,7.5,24.0,3.6,24.0,6.0,999.9,43.0,32.0,0.09G,999.9,110000
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,0002,-8.0,12.0,0014,280.0,279.0,263.0,2330,0737,4.0,0750,0741,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,LAX,722950-23174,PBI,722030-12844,722950-23174,2015-01-01,47.1,24.0,22.5,24.0,1018.8,24.0,1007.2,24.0,10.0,24.0,5.5,24.0,15.0,999.9,57.0,36.0,0.00G,999.9,000000
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,0018,-2.0,16.0,0034,286.0,293.0,266.0,2296,0800,11.0,0806,0811,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,SFO,724940-23234,CLT,723140-13881,724940-23234,2015-01-01,49.5,24.0,30.9,24.0,1019.4,24.0,1018.8,24.0,10.0,24.0,7.8,24.0,15.0,999.9,57.9,39.9,0.00G,999.9,000000
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,0015,-5.0,15.0,0030,285.0,281.0,258.0,2342,0748,8.0,0805,0756,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,LAX,722950-23174,MIA,722020-12839,722950-23174,2015-01-01,47.1,24.0,22.5,24.0,1018.8,24.0,1007.2,24.0,10.0,24.0,5.5,24.0,15.0,999.9,57.0,36.0,0.00G,999.9,000000
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,0024,-1.0,11.0,0035,235.0,215.0,199.0,1448,0254,5.0,0320,0259,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,SEA,727930-24233,ANC,702725-26491,727930-24233,2015-01-01,32.9,24.0,22.2,24.0,1029.7,24.0,1012.7,24.0,10.0,24.0,4.6,24.0,11.1,999.9,42.1,26.1,0.00G,999.9,000000


### Rename merged columns
Columns need to be renamed to prevent issues when merging the dataframes again.

In [16]:
flights_merged_df = flights_merged_df.drop(['STATION', 'YEARMODA'], 1)
flights_merged_df = flights_merged_df.rename(columns={'TEMP': 'OR_TEMP', 'COUNT_TEMP': 'OR_COUNT_TEMP', 'DEWP': 'OR_DEWP', 'COUNT_DEWP': 'OR_COUNT_DEWP', 'SLP': 'OR_SLP', 'COUNT_SLP': 'OR_COUNT_SLP', 'STP': 'OR_STP', 'COUNT_STP': 'OR_COUNT_STP', 'VISIB': 'OR_VISIB', 'COUNT_VISIB': 'OR_COUNT_VISIB', 'WDSP': 'OR_WDSP', 'COUNT_WDSP': 'OR_COUNT_WDSP', 'MXSPD': 'OR_MXSPD', 'GUST': 'OR_GUST', 'MAX': 'OR_MAX', 'MIN': 'OR_MIN', 'PRCP': 'OR_PRCP', 'SNDP': 'OR_SNDP', 'FRSHTT': 'OR_FRSHTT'})

### Merge the weather for destination airport
Merge on matching date and station. Also rename columns to match format of origin weather data

In [20]:
flights_merged_df = pd.merge(flights_merged_df, weather_df, left_on=['destination_weather_station', 'DATE'], right_on=['STATION', 'YEARMODA'], how='left')
flights_merged_df.head()
flights_merged_df = flights_merged_df.drop(['STATION', 'YEARMODA'], 1)
flights_merged_df = flights_merged_df.rename(columns={'TEMP': 'DES_TEMP', 'COUNT_TEMP': 'DES_COUNT_TEMP', 'DEWP': 'DES_DEWP', 'COUNT_DEWP': 'DES_COUNT_DEWP', 'SLP': 'DES_SLP', 'COUNT_SLP': 'DES_COUNT_SLP', 'STP': 'DES_STP', 'COUNT_STP': 'DES_COUNT_STP', 'VISIB': 'DES_VISIB', 'COUNT_VISIB': 'DES_COUNT_VISIB', 'WDSP': 'DES_WDSP', 'COUNT_WDSP': 'DES_COUNT_WDSP', 'MXSPD': 'DES_MXSPD', 'GUST': 'DES_GUST', 'MAX': 'DES_MAX', 'MIN': 'DES_MIN', 'PRCP': 'DES_PRCP', 'SNDP': 'DES_SNDP', 'FRSHTT': 'DES_FRSHTT'})
flights_merged_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,2354,-11.0,21.0,0015,205.0,194.0,169.0,1448,0404,4.0,0430,0408,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233,35.1,24.0,30.7,24.0,1020.1,14.0,1015.2,24.0,7.5,24.0,3.6,24.0,6.0,999.9,43.0,32.0,0.09G,999.9,110000,32.9,24.0,22.2,24.0,1029.7,24.0,1012.7,24.0,10.0,24.0,4.6,24.0,11.1,999.9,42.1,26.1,0.00G,999.9,000000
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,0002,-8.0,12.0,0014,280.0,279.0,263.0,2330,0737,4.0,0750,0741,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,LAX,722950-23174,PBI,722030-12844,47.1,24.0,22.5,24.0,1018.8,24.0,1007.2,24.0,10.0,24.0,5.5,24.0,15.0,999.9,57.0,36.0,0.00G,999.9,000000,74.1,24.0,69.7,24.0,1021.1,22.0,1020.5,24.0,9.1,24.0,5.5,24.0,9.9,999.9,79.0,70.0,0.33G,999.9,010000
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,0018,-2.0,16.0,0034,286.0,293.0,266.0,2296,0800,11.0,0806,0811,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,SFO,724940-23234,CLT,723140-13881,49.5,24.0,30.9,24.0,1019.4,24.0,1018.8,24.0,10.0,24.0,7.8,24.0,15.0,999.9,57.9,39.9,0.00G,999.9,000000,38.4,24.0,28.4,24.0,1026.8,24.0,998.5,24.0,9.7,24.0,2.3,24.0,7.0,999.9,53.1,26.1,0.00G,999.9,000000
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,0015,-5.0,15.0,0030,285.0,281.0,258.0,2342,0748,8.0,0805,0756,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,LAX,722950-23174,MIA,722020-12839,47.1,24.0,22.5,24.0,1018.8,24.0,1007.2,24.0,10.0,24.0,5.5,24.0,15.0,999.9,57.0,36.0,0.00G,999.9,000000,74.8,24.0,67.8,24.0,1021.3,24.0,1020.2,24.0,8.9,24.0,4.1,24.0,11.1,999.9,82.0,70.0,0.00G,999.9,000000
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,0024,-1.0,11.0,0035,235.0,215.0,199.0,1448,0254,5.0,0320,0259,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,SEA,727930-24233,ANC,702725-26491,32.9,24.0,22.2,24.0,1029.7,24.0,1012.7,24.0,10.0,24.0,4.6,24.0,11.1,999.9,42.1,26.1,0.00G,999.9,000000,35.1,24.0,30.7,24.0,1020.1,14.0,1015.2,24.0,7.5,24.0,3.6,24.0,6.0,999.9,43.0,32.0,0.09G,999.9,110000


### Save merged data

In [21]:
flights_merged_df.to_csv(flights_weather_path)